# Dependencies

In [1]:
%pip -q install "pydantic<2.12" "openai>=1.40.0" "litellm>=1.43.0" "python-dotenv>=1.0.1"

# %pip -q install gradio

import os
# os.environ["OPENAI_API_KEY"] = ""
# os.environ["GEMINI_API_KEY"] = ""

Note: you may need to restart the kernel to use updated packages.


In [2]:
from openai import OpenAI
client = OpenAI()

In [3]:
from pydantic import BaseModel, Field
from typing import Literal
from openai import OpenAI

client = OpenAI()

class Sentiment(BaseModel):
  label: Literal["positive", "neutral", "negative", "unsure"]
  positiveness: float = Field(ge=1.0, le=10.0)
  confidence: float = Field(ge=1.0, le=10.0)
  rationale: str

SYSTEM = (
    "You are a sentiment analyst. "
    "Return exclusively the solicited JSON."
)


In [4]:
text_to_analyze = "I loved the service, but they were kind of late."


In [5]:
from litellm import completion
import json

def analyze_sentiment_litellm(text: str, model: str = "gpt-5-nano"):
    messages = [
        {"role": "system", "content": SYSTEM},
        {"role": "user", "content": f"Texto:\n{text}\n\nReturn JSON : label (positive/neutral/negative/unsure), positiveness [1, 10], confidence [1, 10], rationale (very short explanation)."}
    ]
    out = completion(
        model=model, 
        messages=messages, 
        response_format={"type": "json_object"},
        # service_tier="flex"
    )
    print(f'Out: {out}')
    print(out.choices)
    payload = json.loads(out.choices[0].message["content"])

    return Sentiment(**payload)    


In [6]:
analyze_sentiment_litellm(text_to_analyze)

Out: ModelResponse(id='chatcmpl-CY1Le6jMGP9V3tk9ttElTCkKdpGrY', created=1762224370, model='gpt-5-nano-2025-08-07', object='chat.completion', system_fingerprint=None, choices=[Choices(finish_reason='stop', index=0, message=Message(content='{\n  "label": "positive",\n  "positiveness": 7,\n  "confidence": 7,\n  "rationale": "Overall positive; service praised despite lateness."\n}', role='assistant', tool_calls=None, function_call=None, provider_specific_fields={'refusal': None}, annotations=[]), provider_specific_fields={})], usage=Usage(completion_tokens=880, prompt_tokens=72, total_tokens=952, completion_tokens_details=CompletionTokensDetailsWrapper(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=832, rejected_prediction_tokens=0, text_tokens=None), prompt_tokens_details=PromptTokensDetailsWrapper(audio_tokens=0, cached_tokens=0, text_tokens=None, image_tokens=None)), service_tier='default')
[Choices(finish_reason='stop', index=0, message=Message(content='{\n  "label": "p

Sentiment(label='positive', positiveness=7.0, confidence=7.0, rationale='Overall positive; service praised despite lateness.')